In [1]:
%%bash --bg
# Cell 1: launch ROS master + sim time
roscore & sleep 3
rosparam set /use_sim_time true   # must be before any node :contentReference[oaicite:2]{index=2}


In [2]:
%%bash --bg
# Cell 2: Gazebo TurtleBot3 world (waffle)
export TURTLEBOT3_MODEL=waffle
roslaunch turtlebot3_gazebo turtlebot3_world.launch use_sim_time:=true   # :contentReference[oaicite:3]{index=3}


In [3]:
%%bash --bg
# Cell 3: serve the static map
rosrun map_server map_server \
  "$(rospack find turtlebot3_navigation)/maps/map.yaml" \
  _use_sim_time:=true   # 


In [4]:
%%bash --bg
# Cell 4: optional keyboard teleop
export TURTLEBOT3_MODEL=waffle
roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch  # :contentReference[oaicite:4]{index=4}


In [5]:
# Cell 5: AutonomousNavigator (with fixed occupancy check)
import rospy, random, math
from nav_msgs.msg import OccupancyGrid, Odometry
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist
import tf.transformations as tft

class AutonomousNavigator:
    def __init__(self):
        rospy.init_node('autonomous_navigator', anonymous=False)  # no use_sim_time arg :contentReference[oaicite:5]{index=5}
        rospy.loginfo("Node started")
        rospy.loginfo("Waiting for /map...")
        self.map_msg = rospy.wait_for_message('/map', OccupancyGrid)
        rospy.loginfo("Map received")
        self.width  = self.map_msg.info.width
        self.height = self.map_msg.info.height
        self.res    = self.map_msg.info.resolution
        self.origin = self.map_msg.info.origin.position
        self.occ    = list(self.map_msg.data)
        self.cmd_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        rospy.Subscriber('/scan', LaserScan, self.check_and_avoid)
        rospy.Subscriber('/odom', Odometry, self.odom_callback)
        self.odom = None
        self.avoiding = False
        self.rate = rospy.Rate(5)

    def odom_callback(self, msg):
        self.odom = msg.pose.pose

    def compute_path(self, start, goal):
        """A* on 8‐connected grid, only allowing truly free cells (0)."""
        def h(a,b): return math.hypot(a[0]-b[0], a[1]-b[1])
        dirs = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
        open_set = {start: 0}
        came_from = {}
        g_score = {start: 0}
        f_score = {start: h(start,goal)}
        while open_set:
            current = min(open_set, key=open_set.get)
            if current == goal:
                # reconstruct
                path=[]
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                return path[::-1]
            open_set.pop(current)
            for dx, dy in dirs:
                nbx, nby = current[0] + dx, current[1] + dy
                if not (0 <= nbx < self.width and 0 <= nby < self.height):
                    continue
                idx = nby * self.width + nbx
                if self.occ[idx] != 0:
                    continue
                tentative = g_score[current] + math.hypot(dx, dy)
    # Correct membership check using a tuple key:
                if (nbx, nby) not in g_score or tentative < g_score[(nbx, nby)]:
                    came_from[(nbx, nby)] = current
                    g_score[(nbx, nby)]     = tentative
                    f_score[(nbx, nby)]     = tentative + h((nbx, nby), goal)
                    open_set[(nbx, nby)]    = f_score[(nbx, nby)]

        return []

    def check_and_avoid(self, scan):
        if not self.odom: return
        ang_min,inc = scan.angle_min, scan.angle_increment
        center = int((0-ang_min)/inc)
        span   = int(math.radians(20)/inc)
        sector = scan.ranges[max(0,center-span): center+span]
        if sector and min(sector) < 0.5:
            cmd=Twist()
            cmd.angular.z = random.choice([-0.6,0.6])
            self.cmd_pub.publish(cmd)
            self.avoiding = True

    def goto_waypoint(self, wp):
        if not self.odom: return False
        px,py = self.odom.position.x, self.odom.position.y
        ori   = self.odom.orientation
        _,_,yaw = tft.euler_from_quaternion([ori.x,ori.y,ori.z,ori.w])
        goal_yaw = math.atan2(wp[1]-py, wp[0]-px)
        err = (goal_yaw-yaw+math.pi)%(2*math.pi)-math.pi
        cmd=Twist()
        if abs(err)>0.1:
            cmd.angular.z = 0.5*(1 if err>0 else -1)
        else:
            dist = math.hypot(wp[0]-px, wp[1]-py)
            if dist>0.1: cmd.linear.x=0.2
            else: return True
        self.cmd_pub.publish(cmd)
        return False

    def world_to_grid(self, x,y):
        return (int((x-self.origin.x)/self.res),
                int((y-self.origin.y)/self.res))

    def start(self):
        while not self.odom and not rospy.is_shutdown():
            self.rate.sleep()
        # use centre of map for goal
        goal = (self.width//2, self.height//2)
        start = self.world_to_grid(self.odom.position.x,
                                   self.odom.position.y)
        rospy.loginfo(f"Planning from {start} to {goal}")
        path = self.compute_path(start, goal)
        rospy.loginfo(f"Path length: {len(path)} waypoints")
        idx = 0
        while not rospy.is_shutdown() and idx < len(path):
            if self.avoiding:
                self.rate.sleep()
                continue
            gx,gy = path[idx]
            wx = gx*self.res + self.origin.x + self.res/2
            wy = gy*self.res + self.origin.y + self.res/2
            if self.goto_waypoint((wx,wy)):
                idx+=1
                rospy.loginfo(f"Reached {idx}/{len(path)}")
            self.rate.sleep()


In [6]:
# Instantiate the navigator so 'nav' exists in this kernel
nav = AutonomousNavigator()


[INFO] [1745705222.302887, 0.000000]: Node started
[INFO] [1745705222.305185, 0.000000]: Waiting for /map...
[INFO] [1745705222.452842, 14.799000]: Map received


In [7]:
# Cell 6: debug grid indices & occupancy
# assume nav = AutonomousNavigator() has been instantiated
start = nav.world_to_grid(nav.odom.position.x, nav.odom.position.y)
goal  = (nav.width//2, nav.height//2)  # centre of map

start_idx = start[1]*nav.width + start[0]
goal_idx  = goal[1]*nav.width + goal[0]

print(f"Start grid: {start}, occ value: {nav.occ[start_idx]}")
print(f"Goal  grid: {goal }, occ value: {nav.occ[ goal_idx]}")

test_path = nav.compute_path(start, goal)
print("Test path:", test_path)


Start grid: (159, 190), occ value: 0
Goal  grid: (192, 192), occ value: 0
Test path: [(160, 190), (161, 190), (162, 190), (163, 190), (164, 190), (165, 190), (166, 190), (167, 190), (168, 190), (169, 190), (170, 190), (171, 190), (172, 190), (173, 190), (174, 190), (175, 190), (176, 190), (177, 190), (178, 190), (179, 190), (180, 190), (181, 190), (182, 190), (183, 190), (184, 190), (185, 190), (186, 190), (187, 190), (188, 190), (189, 191), (190, 191), (191, 191), (192, 192)]


In [8]:
# Cell 7: start navigation
if __name__ == '__main__':
    nav = AutonomousNavigator()
    nav.start()


[INFO] [1745705263.319690, 37.054000]: Node started
[INFO] [1745705263.335950, 37.069000]: Waiting for /map...
[INFO] [1745705263.511749, 37.165000]: Map received


[INFO] [1745705263.960540, 37.386000]: Planning from (160, 190) to (192, 192)
[INFO] [1745705263.989563, 37.399000]: Path length: 32 waypoints
[INFO] [1745705265.591570, 37.986000]: Reached 1/32
[INFO] [1745705271.345654, 39.986000]: Reached 2/32
[INFO] [1745705273.473290, 40.786000]: Reached 3/32
[INFO] [1745705276.754740, 41.989000]: Reached 4/32
[INFO] [1745705280.546742, 43.387000]: Reached 5/32
[INFO] [1745705282.292858, 43.986000]: Reached 6/32
[INFO] [1745705285.185421, 45.186000]: Reached 7/32
[INFO] [1745705288.480567, 46.386000]: Reached 8/32
[INFO] [1745705288.996281, 46.586000]: Reached 9/32
[INFO] [1745705289.442341, 46.786000]: Reached 10/32
[INFO] [1745705290.329969, 47.186000]: Reached 11/32
[INFO] [1745705290.762758, 47.391000]: Reached 12/32
[INFO] [1745705291.321111, 47.586000]: Reached 13/32
[INFO] [1745705291.839249, 47.786000]: Reached 14/32
[INFO] [1745705292.745815, 48.186000]: Reached 15/32
[INFO] [1745705293.126734, 48.386000]: Reached 16/32
[INFO] [1745705293

KeyboardInterrupt: 

In [ ]:
!rostopic list | grep -E "/map|/scan|/odom|/cmd_vel|/clock"
